## Implement MLFLOW With Deep learning project (using simpleRNN)

###### Run hyperparameter sweep on a trainig script  
###### choose the best run and registerd it is as a model
###### Comapare the result of the various run in the mlflow UI
###### Deploy the model to a REST API
###### Build a container image suitable for deployment to cloud platform


In [3]:
import keras
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [6]:
data=pd.read_csv("../2-DLMLFLOW/data/wine-quality.csv")

In [10]:
data.head(50)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,6.2,0.32,0.16,7.00,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,8.1,0.22,0.43,1.50,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [11]:
train,test = train_test_split(data,test_size=0.25,random_state=42)

In [12]:
train


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [ ]:
##  dprepaire the dataset for training the model


train_x=train.drop([ "quality"],axis=1).values
train_y=train[["quality"]].values.ravel()

In [17]:
train_y

array([6, 6, 5, ..., 6, 6, 8], shape=(3673,))

In [18]:
train_x

array([[ 6.3 ,  0.25,  0.22, ...,  3.16,  0.5 , 10.5 ],
       [ 7.8 ,  0.3 ,  0.29, ...,  3.16,  0.38,  9.  ],
       [ 7.4 ,  0.38,  0.27, ...,  3.17,  0.43, 10.  ],
       ...,
       [ 7.6 ,  0.27,  0.52, ...,  3.02,  0.53, 11.4 ],
       [ 6.3 ,  0.24,  0.29, ...,  3.17,  0.38, 10.6 ],
       [ 8.1 ,  0.27,  0.35, ...,  3.22,  0.63, 10.4 ]], shape=(3673, 11))

In [ ]:
##  dprepaire the dataset for test of the model

test_x = test.drop([ "quality"],axis=1).values
test_y = test[["quality"]].values.ravel()

In [20]:
test_x

array([[ 6.        ,  0.29      ,  0.41      , ...,  3.09      ,
         0.59      , 10.96666667],
       [ 5.4       ,  0.53      ,  0.16      , ...,  3.2       ,
         0.53      , 13.2       ],
       [ 7.1       ,  0.25      ,  0.39      , ...,  3.28      ,
         0.43      , 12.2       ],
       ...,
       [ 5.9       ,  0.13      ,  0.28      , ...,  3.43      ,
         0.64      , 10.8       ],
       [ 6.8       ,  0.3       ,  0.29      , ...,  3.03      ,
         0.32      , 12.9       ],
       [ 7.4       ,  0.25      ,  0.36      , ...,  3.19      ,
         0.44      , 10.8       ]], shape=(1225, 11))

In [21]:
test_y

array([7, 8, 8, ..., 6, 7, 6], shape=(1225,))

In [23]:
## taking the train data again and spliting into validation and training 

train_x,valid_x,train_y , valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

In [24]:
signature = infer_signature(train_x,train_y)

In [26]:
mean = np.mean(train_x,axis = 0)
mean

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [ ]:
## we nee to create our ANN model 

def train_model(params,epoch,train_x,train_y,valid_x,valid_y,test_x,test_y):
    ## model architecture 
    mean = np.mean(train_x,axis = 0)
    var = np.var(train_x,axis = 0)
    
    model = keras.Sequential(
        
        [keras.input ( train_x.shape[1])
         
         ]     
    )
    